<a href="https://colab.research.google.com/github/jwlw2022/nlp-chatbot-project/blob/main/6864_NLP_Chatbot_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
# %%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
!pip install pytorch-pretrained-bert
!pip -q install transformers
!pip -q install datasets
!pip -q install tqdm
!pip -q install sentencepiece 

     |████████████████████████████████| 133kB 13.6MB/s 
     |████████████████████████████████| 133kB 27.2MB/s 
     |████████████████████████████████| 7.5MB 14.2MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
ERROR: botocore 1.20.62 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 2.1MB 14.8MB/s 
     |████████████████████████████████| 3.3MB 58.4MB/s 
     |████████████████████████████████| 901kB 59.0MB/s 
     |████████████████████████████████| 225kB 12.8MB/s 
     |████████████████████████████████| 112kB 22.1MB/s 
     |████████████████████████████████| 245kB 18.9MB/s 
     |████████████████████████████████| 1.2MB 16.5MB/s 


# Pretrained tokenizer

In [ ]:
import transformers

# Use a pretrained tokenizer with CLASS.from_pretrained() function
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-cased')

# Download PersonaChat dataset


In [ ]:
import json
from pytorch_pretrained_bert import cached_path

url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"

# Download and load JSON dataset
personachat_file = cached_path(url)
with open(personachat_file, "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())

# Tokenize and encode the dataset using our loaded GPT tokenizer
def tokenize(obj):
    if isinstance(obj, str):
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
    if isinstance(obj, dict):
        return dict((n, tokenize(o)) for n, o in obj.items())
    return list(tokenize(o) for o in obj)
 
dataset = tokenize(dataset)

100%|██████████| 209850483/209850483 [00:08<00:00, 24423625.76B/s]


In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim

In [ ]:
def make_optimizer(model):
    # Get all the parameters
    params_to_update = model.parameters()
    print("Params to learn:")
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

    # Use SGD
    optimizer = optim.Adam(params_to_update)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)
    return optimizer, scheduler

# Seq2Seq Code

## Download Dataset

In [24]:
%%bash
git clone https://github.com/mit-6864/hw3.git
mkdir -p /content/hw3/data

pip install sacrebleu

fatal: destination path 'hw3' already exists and is not an empty directory.


In [15]:
# Download data
DATA_DIR = "/content/hw3/data"

!wget -nv -O "$DATA_DIR/train.en" https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en
!wget -nv -O "$DATA_DIR/train.vi" https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.vi
!wget -nv -O "$DATA_DIR/vocab.en" https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.en
!wget -nv -O "$DATA_DIR/vocab.vi" https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.vi

2021-05-01 00:16:19 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en [13603614/13603614] -> "/content/hw3/data/train.en" [1]
2021-05-01 00:17:09 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.vi [18074646/18074646] -> "/content/hw3/data/train.vi" [1]
2021-05-01 00:17:10 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2013.en [132264/132264] -> "/content/hw3/data/tst2013.en" [1]
2021-05-01 00:17:12 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2013.vi [183855/183855] -> "/content/hw3/data/tst2013.vi" [1]
2021-05-01 00:17:14 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.en [139741/139741] -> "/content/hw3/data/vocab.en" [1]
2021-05-01 00:17:15 URL:https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.vi [46767/46767] -> "/content/hw3/data/vocab.vi" [1]


In [37]:
import sys
sys.path.append("/content/hw3")
import lab_utils

from lab_utils import read_vocab_file, read_sentence_file, filter_data, show_some_data_stats

src_vocab_set = read_vocab_file("vocab.vi")
trg_vocab_set = read_vocab_file("vocab.en")

train_src_sentences_list = read_sentence_file("train.vi")
train_trg_sentences_list = read_sentence_file("train.en")
assert len(train_src_sentences_list) == len(train_trg_sentences_list)

test_src_sentences_list = read_sentence_file("tst2013.vi")
test_trg_sentences_list = read_sentence_file("tst2013.en")
assert len(test_src_sentences_list) == len(test_trg_sentences_list)

# Filter out sentences over 48 words long
MAX_SENT_LENGTH = 48
MAX_SENT_LENGTH_PLUS_SOS_EOS = 50

train_src_sentences_list, train_trg_sentences_list = filter_data(
    train_src_sentences_list, train_trg_sentences_list, MAX_SENT_LENGTH)
test_src_sentences_list, test_trg_sentences_list = filter_data(
    test_src_sentences_list, test_trg_sentences_list, MAX_SENT_LENGTH)

# We take 10% of training data as validation set.
num_val = int(len(train_src_sentences_list) * 0.1)
val_src_sentences_list = train_src_sentences_list[:num_val]
val_trg_sentences_list = train_trg_sentences_list[:num_val]
train_src_sentences_list = train_src_sentences_list[num_val:]
train_trg_sentences_list = train_trg_sentences_list[num_val:]

# show_some_data_stats(train_src_sentences_list, val_src_sentences_list, 
                    #  test_src_sentences_list, train_trg_sentences_list,
                    #  src_vocab_set, trg_vocab_set)

In [38]:
import torch
from torch.utils import data

# These IDs are reserved.
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3


class MTDataset(data.Dataset):
  def __init__(self, src_sentences, src_vocabs, trg_sentences, trg_vocabs,
               sampling=1.):
    self.src_sentences = src_sentences[:int(len(src_sentences) * sampling)]
    self.trg_sentences = trg_sentences[:int(len(src_sentences) * sampling)]

    self.max_src_seq_length = MAX_SENT_LENGTH_PLUS_SOS_EOS
    self.max_trg_seq_length = MAX_SENT_LENGTH_PLUS_SOS_EOS

    self.src_vocabs = src_vocabs
    self.trg_vocabs = trg_vocabs

    self.src_v2id = {v : i for i, v in enumerate(src_vocabs)}
    self.src_id2v = {val : key for key, val in self.src_v2id.items()}
    self.trg_v2id = {v : i for i, v in enumerate(trg_vocabs)}
    self.trg_id2v = {val : key for key, val in self.trg_v2id.items()}

  def __len__(self):
    return len(self.src_sentences)

  def __getitem__(self, index):
    src_sent = self.src_sentences[index]
    src_len = len(src_sent) + 2   # add <s> and </s> to each sentence
    src_id = []
    for w in src_sent:
      if w not in self.src_vocabs:
        w = '<unk>'
      src_id.append(self.src_v2id[w])
    src_id = ([SOS_INDEX] + src_id + [EOS_INDEX] + [PAD_INDEX] *
              (self.max_src_seq_length - src_len))

    trg_sent = self.trg_sentences[index]
    trg_len = len(trg_sent) + 2
    trg_id = []
    for w in trg_sent:
      if w not in self.trg_vocabs:
        w = '<unk>'
      trg_id.append(self.trg_v2id[w])
    trg_id = ([SOS_INDEX] + trg_id + [EOS_INDEX] + [PAD_INDEX] *
              (self.max_trg_seq_length - trg_len))

    return torch.tensor(src_id), src_len, torch.tensor(trg_id), trg_len

## Encoder

In [39]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, dropout=0.):
    """
    Inputs: 
      - `input_size`: an int representing the RNN input size.
      - `hidden_size`: an int representing the RNN hidden size.
      - `dropout`: a float representing the dropout rate during training. Note
          that for 1-layer RNN this has no effect since dropout only applies to
          outputs of intermediate layers.
    """
    
    super(Encoder, self).__init__()

    # --------- Your code here --------- #
    # feel free to use a pre-implemented pytorch GRU
    # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

    self.rnn = nn.GRU(input_size=input_size,
                      hidden_size=hidden_size,
                      num_layers=3,
                      batch_first=True,
                      dropout=dropout)
    
    # --------- Your code ends --------- #

  def forward(self, inputs, lengths):
    """
    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of source
          sentences.
      - `lengths`: a 1d-tensor of shape (batch_size,) representing the sequence
          lengths of `inputs`.

    Returns:
      - `outputs`: a 3d-tensor of shape
        (batch_size, max_seq_length, hidden_size).
      - `finals`: a 3d-tensor of shape (num_layers, batch_size, hidden_size).

      Hint: `outputs` and `finals` are both standard GRU outputs.
    """
    outputs = None
    finals = None
    
    # --------- Your code here --------- #
    # hint: you probably want to pack the inputs and outputs (see note below)
    #       https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html
    # hint2: given the shape of the inputs and outputs, 
    #        it might be helpful to specify batch_first=True (also in __init___)
    # hint3: MAX_SENT_LENGTH_PLUS_SOS_EOS is a global variable that exists if 
    #        you ever need to specify a total_length for outputs

    packed_sequence = pack_padded_sequence(inputs,
                                           lengths.cpu(),
                                           batch_first=True,
                                           enforce_sorted=False)
    outputs, finals = self.rnn(packed_sequence)
    outputs, _ = pad_packed_sequence(outputs, batch_first=True)
    
    # --------- Your code ends --------- #

    return outputs, finals

## Decoder

In [40]:
class Decoder(nn.Module):
  """An RNN decoder without attention."""

  def __init__(self, input_size, hidden_size, dropout=0.):
    """
      Inputs:
        - `input_size`, `hidden_size`, and `dropout` the same as in Encoder.
    """
    super(Decoder, self).__init__()

    # --------- Your code here --------- #
    # hint: you need more layers than the encoder
    #       again, feel free to use pytorch implemetnations
    #       https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
    
    # To initialize from the final encoder state.

    self.rnn = nn.GRU(input_size=input_size,
                      hidden_size=hidden_size,
                      num_layers=3,
                      batch_first=True,
                      dropout=dropout)

    self.bridge = nn.Linear(hidden_size, input_size, bias=True)

    # --------- Your code ends --------- #

  def forward_step(self, prev_embed, hidden):
    """Helper function for forward below:
       Perform a single decoder step (1 word).

       Inputs:
      - `prev_embed`: a 3d-tensor of shape (batch_size, 1, embed_size)
          representing the padded embedded word vectors at this step in training
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the current hidden state.

      Returns:
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
          representing the current decoder hidden state.
      - `pre_output`: a 3d-tensor of shape (batch_size, 1, hidden_size)
          representing the total decoder output for one step
    """
    pre_output = None
    # --------- Your code here --------- #
    
    pre_output, hidden = self.rnn(prev_embed, hidden)

    # --------- Your code ends --------- #
    return hidden, pre_output

  def forward(self, inputs, encoder_finals, hidden=None, max_len=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of target
          sentences (for teacher-forcing during training).
      - `encoder_finals`: a 3d-tensor of shape
          (num_enc_layers, batch_size, hidden_size) representing the final
          encoder hidden states used to initialize the initial decoder hidden
          states.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the value to be used to initialize the initial decoder hidden states.
          If None, then use `encoder_finals`.
      - `max_len`: an int representing the maximum decoding length.

    Returns:
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_seq_length, hidden_size) representing the raw
          decoder outputs (before converting to a `trg_vocab_size`-dim vector).
          We will convert it later in a `Generator` below.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
          representing the last decoder hidden state.
    """

    # The maximum number of steps to unroll the RNN.
    if max_len is None:
      max_len = inputs.size(1)

    # Initialize decoder hidden state.
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)

    outputs = []
    
    # --------- Your code here --------- #

    # Unroll the decoder RNN for `max_len` steps.
    # hint: use the above helper function forward_step that 
    #       performs a single decoder step (1 word).

    for i in range(max_len):
        prev_embed = inputs[:, i:i+1, :] # get embeddings from inputs
        hidden, pre_output = self.forward_step(prev_embed, hidden)
        outputs.append(pre_output)
    
    # final output concatenates outputs for each word
    outputs = torch.cat(outputs, dim=1)

    # --------- Your code ends --------- #

    return hidden, outputs

  def init_hidden(self, encoder_finals):
    """Use encoder final hidden state to initialize decoder's first hidden
    state."""
    decoder_init_hiddens = torch.tanh(self.bridge(encoder_finals))

    return decoder_init_hiddens

## EncoderDecoder

In [41]:
class EncoderDecoder(nn.Module):
  """A standard Encoder-Decoder architecture without attention.
  """
  def __init__(self, encoder, decoder, src_embed, trg_embed, generator):
    """
    Inputs:
      - `encoder`: an `Encoder` object.
      - `decoder`: a `Decoder` object.
      - `src_embed`: an nn.Embedding object representing the lookup table for
          input (source) sentences.
      - `trg_embed`: an nn.Embedding object representing the lookup table for
          output (target) sentences.
      - `generator`: a `Generator` object. Essentially a linear mapping. See
          the next code cell.
    """
    super(EncoderDecoder, self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.trg_embed = trg_embed
    self.generator = generator

  def forward(self, src_ids, trg_ids, src_lengths):
    """Take in and process masked source and target sequences.

    Inputs:
      `src_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
        a batch of source sentences of word ids.
      `trg_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
        a batch of target sentences of word ids.
      `src_lengths`: a 1d-tensor of shape (batch_size,) representing the
        sequence length of `src_ids`.

    Returns the decoder outputs, see the above cell.
    """
    encoder_hiddens, encoder_finals = self.encode(src_ids, src_lengths)
    del encoder_hiddens   # unused
    return self.decode(encoder_finals, trg_ids[:, :-1])

  def encode(self, src_ids, src_lengths):
    return self.encoder(self.src_embed(src_ids), src_lengths)
    
  def decode(self, encoder_finals, trg_ids, decoder_hidden=None):
    return self.decoder(self.trg_embed(trg_ids), encoder_finals, decoder_hidden)

## Generator

In [42]:
class Generator(nn.Module):
  """Define standard linear + softmax generation step."""
  def __init__(self, hidden_size, vocab_size):
    super(Generator, self).__init__()
    self.proj = nn.Linear(hidden_size, vocab_size, bias=False)

  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

## Training loop

In [43]:
from torch.utils import data

batch_size = 128

# You can try on a smaller training set by setting a smaller `sampling`.
train_set = MTDataset(train_src_sentences_list, src_vocab_set,
                      train_trg_sentences_list, trg_vocab_set, sampling=1.)
train_data_loader = data.DataLoader(train_set, batch_size=batch_size,
                                    num_workers=8, shuffle=True)

val_set = MTDataset(val_src_sentences_list, src_vocab_set,
                    val_trg_sentences_list, trg_vocab_set, sampling=1.)
val_data_loader = data.DataLoader(val_set, batch_size=batch_size, num_workers=8,
                                  shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [44]:
import math


class SimpleLossCompute:
  """A simple loss compute and train function."""

  def __init__(self, generator, criterion, opt=None):
    self.generator = generator
    self.criterion = criterion
    self.opt = opt

  def __call__(self, x, y, norm):
    x = self.generator(x)
    loss = self.criterion(x.contiguous().view(-1, x.size(-1)),
                          y.contiguous().view(-1))
    loss = loss / norm

    if self.opt is not None:  # training mode
      loss.backward()          
      self.opt.step()
      self.opt.zero_grad()

    return loss.data.item() * norm


def run_epoch(data_loader, model, loss_compute, print_every):
  """Standard Training and Logging Function"""

  total_tokens = 0
  total_loss = 0

  for i, (src_ids_BxT, src_lengths_B, trg_ids_BxL, trg_lengths_B) in enumerate(data_loader):
    # We define some notations here to help you understand the loaded tensor
    # shapes:
    #   `B`: batch size
    #   `T`: max sequence length of source sentences
    #   `L`: max sequence length of target sentences; due to our preprocessing
    #        in the beginning, `L` == `T` == 50
    # An example of `src_ids_BxT` (when B = 2):
    #   [[2, 4, 6, 7, ..., 4, 3, 0, 0, 0],
    #    [2, 8, 6, 5, ..., 9, 5, 4, 3, 0]]
    # The corresponding `src_lengths_B` would be [47, 49].
    # Note that SOS_INDEX == 2, EOS_INDEX == 3, and PAD_INDEX = 0.

    src_ids_BxT = src_ids_BxT.to(device)
    src_lengths_B = src_lengths_B.to(device)
    trg_ids_BxL = trg_ids_BxL.to(device)

    del trg_lengths_B   # unused

    _, output = model(src_ids_BxT, trg_ids_BxL, src_lengths_B)

    loss = loss_compute(x=output, y=trg_ids_BxL[:, 1:],
                        norm=src_ids_BxT.size(0))
    total_loss += loss
    total_tokens += (trg_ids_BxL[:, 1:] != PAD_INDEX).data.sum().item()

    if model.training and i % print_every == 0:
      print("Epoch Step: %d Loss: %f" % (i, loss / src_ids_BxT.size(0)))

  return math.exp(total_loss / float(total_tokens))


def train(model, num_epochs, learning_rate, print_every):
  # Set `ignore_index` as PAD_INDEX so that pad tokens won't be included when
  # computing the loss.
  criterion = nn.NLLLoss(reduction="sum", ignore_index=PAD_INDEX)
  optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Keep track of dev ppl for each epoch.
  dev_ppls = []

  for epoch in range(num_epochs):
    print("Epoch", epoch)

    model.train()
    train_ppl = run_epoch(data_loader=train_data_loader, model=model,
                          loss_compute=SimpleLossCompute(model.generator,
                                                         criterion, optim),
                          print_every=print_every)
        
    model.eval()
    with torch.no_grad():      
      dev_ppl = run_epoch(data_loader=val_data_loader, model=model,
                          loss_compute=SimpleLossCompute(model.generator,
                                                         criterion, None),
                          print_every=print_every)
      print("Validation perplexity: %f" % dev_ppl)
      dev_ppls.append(dev_ppl)
        
  return dev_ppls

In [45]:
device = 'cuda'

# Hyperparameters for contructing the encoder-decoder model.
embed_size = 256   # Each word will be represented as a `embed_size`-dim vector.
hidden_size = 256  # RNN hidden size.
dropout = 0.2

pure_seq2seq = EncoderDecoder(
    encoder=Encoder(embed_size, hidden_size, dropout=dropout),
    decoder=Decoder(embed_size, hidden_size, dropout=dropout),
    src_embed=nn.Embedding(len(src_vocab_set), embed_size),
    trg_embed=nn.Embedding(len(trg_vocab_set), embed_size),
    generator=Generator(hidden_size, len(trg_vocab_set))).to(device)

train_model = True
if train_model:
  # Start training. The returned `dev_ppls` is a list of dev perplexity for each
  # epoch.
  pure_dev_ppls = train(pure_seq2seq, num_epochs=10, learning_rate=1e-3,
                        print_every=100)
  
  torch.save(pure_seq2seq.state_dict(), MODEL_FOLDER+"/" + "pure_seq2seq.pt")

  # Plot perplexity
  lab_utils.plot_perplexity(pure_dev_ppls)
else:
  pure_seq2seq.load_state_dict(torch.load(MODEL_FOLDER+"/" + "pure_seq2seq.pt"))

Epoch 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch Step: 0 Loss: 174.436020
Epoch Step: 100 Loss: 113.114037
Epoch Step: 200 Loss: 102.618744
Epoch Step: 300 Loss: 106.262444
Epoch Step: 400 Loss: 104.514557
Epoch Step: 500 Loss: 98.487244
Epoch Step: 600 Loss: 95.173126
Epoch Step: 700 Loss: 85.836708
Epoch Step: 800 Loss: 84.591370
Validation perplexity: 120.230626
Epoch 1
Epoch Step: 0 Loss: 83.285622
Epoch Step: 100 Loss: 91.261894
Epoch Step: 200 Loss: 80.597115
Epoch Step: 300 Loss: 87.023758
Epoch Step: 400 Loss: 76.764206
Epoch Step: 500 Loss: 76.288330
Epoch Step: 600 Loss: 72.683487
Epoch Step: 700 Loss: 69.780037
Epoch Step: 800 Loss: 81.904305
Validation perplexity: 62.284022
Epoch 2
Epoch Step: 0 Loss: 72.645332
Epoch Step: 100 Loss: 63.379574
Epoch Step: 200 Loss: 76.796875
Epoch Step: 300 Loss: 73.564842
Epoch Step: 400 Loss: 74.326859
Epoch Step: 500 Loss: 73.644722
Epoch Step: 600 Loss: 63.919121
Epoch Step: 700 Loss: 75.219376
Epoch Step: 800 Loss: 67.634926
Validation perplexity: 48.614352
Epoch 3
Epoch Step: 0

NameError: ignored

In [ ]:
def greedy_decode(model, src_ids, src_lengths, max_len):
  """Greedily decode a sentence for EncoderDecoder. Make sure to chop off the 
     EOS token!"""

  with torch.no_grad():
    _, encoder_finals = model.encode(src_ids, src_lengths)
    prev_y = torch.ones(1, 1).fill_(SOS_INDEX).type_as(src_ids)

  output = []
  hidden = None
  
  # --------- Your code here --------- #

  for i in range(max_len):
      with torch.no_grad():
          hidden, out = model.decode(encoder_finals, prev_y, hidden)
          output.append(torch.argmax(model.generator(out)))
          prev_y[0][0] = output[-1]

          if output[-1] == EOS_INDEX:
              output = output[:-1]
              break

  # --------- Your code ends --------- #

  return output